### NLP Setup

In [1]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import the Template

In [2]:
from app.templates.meat_sale.nl_template import nl_template

obs = nl_template['obligations']

print('OBLIGATIONS')
for ob in obs:
    print(f'{ob}: {obs[ob]}\n')

OBLIGATIONS
delivery: The Seller shall deliver the Order in one delivery [DELIVERY_TIMEFRAME] to the Buyer [DELIVERY_LOCATION]

payment: The Buyer shall pay [PAYMENT_DETAILS] to the Seller [PAYMENT_TIMEFRAME].

latePayment: In the event of late payment of the amount owed due, the Buyer shall pay interests equal to [INTEREST_DETAILS]

disclosure: Both Seller and Buyer must keep the contents of this contract confidential [CONFIDENTIALITY_TIMEFRAME]



In [3]:
from app.templates.meat_sale.symboleo.contract_template import get_template

contract_template = get_template()

sym_template = contract_template.to_sym()

sym_template_path = './app/templates/meat_sale/symboleo/symboleo_spec.txt'
with open(sym_template_path, 'w') as f:
    f.write(sym_template)

In [4]:
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

## Setup parms

### Reusable

In [5]:
from app.templates.meat_sale.nl_template import parameters as meat_sale_parms

from app.src.rules.contract_spec.predicate_processor import PredicateProcessor
from app.src.rules.domain_model.domain_prop_processor import DomainPropProcessor

from app.src.norm_proposition_updater import NormPropositionUpdater
from app.classes.spec.predicate_function import PredicateFunctionHappens

from app.classes.spec.sym_point import PointAtomContractEvent, ContractEvent

from app.src.rules.contract_spec.timeframe.timeframe_extractor_builder import TimeFrameExtractorBuilder
from app.src.rules.contract_spec.condition.condition_extractor_builder import ConditionExtractorBuilder

from app.src.rules.domain_model.amount.amount_extractor_builder import AmountExtractorBuilder
from app.src.rules.domain_model.currency.currency_extractor_builder import CurrencyExtractorBuilder
from app.src.rules.domain_model.location.location_extractor_builder import LocationExtractorBuilder

norm_updater = NormPropositionUpdater()

### DELIVERY_TIMEFRAME

In [6]:
delivered_event = contract_template.domain_model.events['delivered'].to_obj()
template = PredicateFunctionHappens(delivered_event)

default_components = [
    PointAtomContractEvent(ContractEvent('activated'))
]
dtf_extractor = TimeFrameExtractorBuilder.build(nlp, template, default_components)

dtf_pred_proc_config = meat_sale_parms['DELIVERY_TIMEFRAME'][0].config

dtf_processor = PredicateProcessor(dtf_pred_proc_config, dtf_extractor, norm_updater)

### DELIVERY_LOCATION

In [7]:
dl_extractor = LocationExtractorBuilder.build(nlp)

dl_config = meat_sale_parms['DELIVERY_LOCATION'][0].config

dl_processor = DomainPropProcessor(dl_config, dl_extractor)

### PAYMENT_DETAILS

In [8]:
pd_currency_extractor = CurrencyExtractorBuilder.build(nlp)

pd_curr_config = meat_sale_parms['PAYMENT_DETAILS'][1].config

pd_curr_processor = DomainPropProcessor(pd_curr_config, pd_currency_extractor)

In [9]:
pd_amount_extractor = AmountExtractorBuilder.build(nlp)

pd_amt_config = meat_sale_parms['PAYMENT_DETAILS'][0].config

pd_amt_processor = DomainPropProcessor(pd_amt_config, pd_amount_extractor)

### PAYMENT_TIMEFRAME

In [10]:
paid_event = contract_template.domain_model.events['paid'].to_obj()
paid_template = PredicateFunctionHappens(paid_event)
default_components = [
    PointAtomContractEvent(ContractEvent('activated'))
]
ptf_extractor = TimeFrameExtractorBuilder.build(nlp, paid_template, default_components)

ptf_config = meat_sale_parms['PAYMENT_TIMEFRAME'][0].config

ptf_processor = PredicateProcessor(ptf_config, ptf_extractor, norm_updater)

### INTEREST_DETAILS

In [11]:
id_currency_extractor = CurrencyExtractorBuilder.build(nlp)

id_curr_config = meat_sale_parms['INTEREST_DETAILS'][1].config

id_curr_processor = DomainPropProcessor(id_curr_config, id_currency_extractor)

In [12]:
id_amount_extractor = AmountExtractorBuilder.build(nlp)

id_amt_config = meat_sale_parms['INTEREST_DETAILS'][0].config

id_amt_processor = DomainPropProcessor(id_amt_config, id_amount_extractor)

### CONFIDENTIALITY_TIMEFRAME

In [13]:
disclosed_event = contract_template.domain_model.events['disclosed'].to_obj()
disclosed_template = PredicateFunctionHappens(disclosed_event)

ctf_extractor = TimeFrameExtractorBuilder.build(nlp, disclosed_template)

ctf_config1 = meat_sale_parms['CONFIDENTIALITY_TIMEFRAME'][0].config

ctf_processor1 = PredicateProcessor(ctf_config1, ctf_extractor, norm_updater)

ctf_config2 = meat_sale_parms['CONFIDENTIALITY_TIMEFRAME'][1].config

ctf_processor2 = PredicateProcessor(ctf_config2, ctf_extractor, norm_updater)

### DELIVERY SUSPENSION CONDITION

In [14]:
suspension_template = None
sc_extractor = ConditionExtractorBuilder.build(nlp, suspension_template)

sc_config = meat_sale_parms['DELIVERY_SUSPENSION_CONDITION'][0].config

sc_processor = PredicateProcessor(sc_config, sc_extractor, norm_updater)

### DELIVERY RESUMPTION CONDITION

In [15]:
from app.classes.spec.sym_interval import Interval, SituationExpression
from app.classes.spec.sym_situation import ObligationState

resumption_template = None
default_components = [
    Interval(SituationExpression(ObligationState('Suspension', 'delivery')))
]
rc_extractor = ConditionExtractorBuilder.build(nlp, resumption_template, default_components)

rc_config = meat_sale_parms['DELIVERY_RESUMPTION_CONDITION'][0].config

rc_processor = PredicateProcessor(rc_config, rc_extractor, norm_updater)

### TERMINATION CONDITION

In [16]:
from app.classes.spec.sym_interval import Interval, SituationExpression
from app.classes.spec.sym_situation import ObligationState

termination_template = None
default_components = [
    ObligationState('Violation', 'delivery')
]
tc_extractor = ConditionExtractorBuilder.build(nlp, termination_template, default_components)

tc_config = meat_sale_parms['TERMINATION_CONDITION'][0].config

tc_processor = PredicateProcessor(tc_config, tc_extractor, norm_updater)

## Execution

### Contract Updater

In [17]:
from app.src.contract_updater import ContractUpdater, ContractUpdateRequest
from app.src.processor_lookup import ProcessorLookup

processor_dict = {
    'DELIVERY_TIMEFRAME': [dtf_processor],
    'DELIVERY_LOCATION': [dl_processor],
    'PAYMENT_DETAILS': [pd_curr_processor, pd_amt_processor], 
    'PAYMENT_TIMEFRAME': [ptf_processor],
    'INTEREST_DETAILS': [id_amt_processor, id_curr_processor],
    'CONFIDENTIALITY_TIMEFRAME': [ctf_processor1, ctf_processor2],
    'DELIVERY_SUSPENSION_CONDITION': [sc_processor],
    'DELIVERY_RESUMPTION_CONDITION': [rc_processor],
    'TERMINATION_CONDITION': [tc_processor]
}
processor_lookup = ProcessorLookup(processor_dict)

contract_updater = ContractUpdater(processor_lookup)

contract = contract_template

In [18]:
customizations = [
    ('DELIVERY_TIMEFRAME', 'within 2 weeks'),
    ('DELIVERY_LOCATION', 'at the buyer\'s warehouse'),
    ('PAYMENT_DETAILS', '$100.00 CAD'),
    ('PAYMENT_TIMEFRAME', 'before April 17, 2022'),
    ('INTEREST_DETAILS', '10% of the amount owed'),
    ('CONFIDENTIALITY_TIMEFRAME', 'until 6 months after termination of the contract'),
    ('DELIVERY_SUSPENSION_CONDITION', 'if payment is not made'),
    ('DELIVERY_RESUMPTION_CONDITION', 'until payment is made'),
    ('TERMINATION_CONDITION', 'unless such delay exceeds 10 days')
]

In [19]:
for c in customizations:
    key = c[0]
    value = c[1]
    doc = nlp(value)

    req = ContractUpdateRequest(contract, key, value, doc)

    print(key)
    contract = contract_updater.update(req)

c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


DELIVERY_TIMEFRAME
DELIVERY_LOCATION


c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_c

[("buyer's warehouse", 0.1), ('buyer.address', 0.042843956273236575), ('seller.address', 0.036577720471448597), ('buyer', 0)]
PAYMENT_DETAILS


c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_c

...

c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_c

[('paid.currency', 0.5920811199370206), ('X', 0)]
CONFIDENTIALITY_TIMEFRAME
DELIVERY_SUSPENSION_CONDITION


c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


DELIVERY_RESUMPTION_CONDITION
TERMINATION_CONDITION


c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [20]:
print(contract.to_sym())


DOMAIN MODEL:
== Roles ==
seller: seller
- address (str): 123 Main street

buyer: buyer
- address (str): 999 Central Ave


== Assets ==
perishableGood: perishableGood (None)
- quantity (number): 
- quality (MeatQuality): 

meat: meat (perishableGood)


== Events ==
delivered: delivered
- item (meat): meat
- location (str): buyer's warehouse

paid: paid
- from (Role): buyer
- to (Role): seller
- currency (str): CAD
- amount (str): 100.00

paidLate: paidLate
- from (Role): buyer
- to (Role): seller
- amount (str): 10% * delivered.location
- currency (str): paid.currency

disclosed: disclosed



CONTRACT SPEC:

== Obligations ==
delivery: O(seller, buyer, true, WhappensBefore(delivered, Date.add(activated(self), 2, weeks)))

payment: O(buyer, seller, true, WhappensBefore(paid, April 17, 2022))

latePayment: Happens(Violated(obligations.payment)) -> O(buyer, seller, true, Happens(paidLate))

disclosure1: O(seller, buyer, true, WhappensBefore(disclosed, Date.add(activated(self), 6, months)